In [2]:
import os

os.environ["SEED"] = "0"
import numpy as np
import pandas as pd
from ConfigSpace import Configuration

from src.aac.AAC import AAC
from src.configuration_space.LKH import CONFIGURATION_SPACE
from src.constant import DATA_DIR, SEED, DATABASE_DIR, MAIN_DIR
from src.database import DB
from src.database.queries import *
from src.instance.Instance import Instance
from src.instance.InstanceList import InstanceList
from src.instance.TSP_Instance import TSP_Instance, TSP_train_test_from_index_file, TSP_from_index_file
from src.solver.Portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
from src.aac.SurrogateEstimator import Estimator1, SurrogateEstimator

In [3]:
test_instances = TSP_from_index_file(
    filepath=DATA_DIR / "TSP" / "TEST_600" / "index.json"
)

In [ ]:
# import joblib
# paths = list((MAIN_DIR / "_archive" / "experiments" / "2025-01-27" / "200" / "database").glob("*.db"))
# for path in paths:
#     db = DB(path)

#     configured_portfolio_solvers = pd.read_sql_query(""" 
#     SELECT DISTINCT solver_id FROM results
#     WHERE prefix LIKE 'test%'                  
#     """, db._conn)["solver_id"].to_list()

#     portfolio = Portfolio.from_iterable([TSP_LKH_Solver.from_db(id_, db) for id_ in configured_portfolio_solvers])
#     joblib.dump(portfolio, MAIN_DIR / "portfolios" / "200" / f"{path.stem}.pkl")

# paths = list((MAIN_DIR / "_archive" / "experiments" / "2025-01-27" / "400" / "database").glob("*.db"))
# for path in paths:
#     db = DB(path)

#     configured_portfolio_solvers = pd.read_sql_query(""" 
#     SELECT DISTINCT solver_id FROM results
#     WHERE prefix LIKE 'test%'                  
#     """, db._conn)["solver_id"].to_list()

#     portfolio = Portfolio.from_iterable([TSP_LKH_Solver.from_db(id_, db) for id_ in configured_portfolio_solvers])
#     joblib.dump(portfolio, MAIN_DIR / "portfolios" / "400" / f"{path.stem}.pkl")

In [ ]:
import joblib

paths = list((MAIN_DIR / "portfolios" / "200").glob("run-plain-30-*.pkl"))
for path in paths:
    run_id = path.stem.split("-")[-1]
    portfolio = joblib.load(path)

    for i in range(100):
        portfolio.evaluate(
            test_instances,
            prefix=f"{run_id};test{i}",
            calculate_features=False,
            cache=False,
        )

[2025-02-10 21:57:41] DEBUG     Portfolio(size=2)[Solver(id=1892086461324458362), Solver(id=737437142660879388)]


[2025-02-10 21:57:41] DEBUG     Portfolio(size=2)[Solver(id=2304390077684693758), Solver(id=1882404664274296862)]
[2025-02-10 21:57:41] DEBUG     Portfolio(size=2)[Solver(id=1584083752273863567), Solver(id=1385232175740324506)]
[2025-02-10 21:57:41] DEBUG     Portfolio(size=2)[Solver(id=1227613603646233546), Solver(id=1366799117327855071)]
[2025-02-10 21:57:41] DEBUG     Portfolio(size=2)[Solver(id=1341915066215952252), Solver(id=47536091475785093)]
[2025-02-10 21:57:42] DEBUG     Portfolio(size=2)[Solver(id=1753552968669763749), Solver(id=1337189405234819855)]
[2025-02-10 21:57:42] DEBUG     Portfolio(size=2)[Solver(id=568744889761080087), Solver(id=1854507948164173178)]
[2025-02-10 21:57:42] DEBUG     Portfolio(size=2)[Solver(id=490255195737833568), Solver(id=249745316546260728)]
[2025-02-10 21:57:42] DEBUG     Portfolio(size=2)[Solver(id=589282855798737225), Solver(id=913893667592655585)]
[2025-02-10 21:57:42] DEBUG     Portfolio(size=2)[Solver(id=2303733698737981910), Solver(id=226